# Разведочный анализ Авто Кредиты (EDA - exploratory data analysis)

Делаем разведочный анализ по Авто Кредитам. Исследуем данные, пытаемся найти инсайты. Строим выводы.

## Подключаем библиотеки

In [ ]:
import pandas as pd
import numpy as np

# игнорируем warnings
import warnings
warnings.filterwarnings("ignore")

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline

# настройка внешнего вида графиков в seaborn
sns.set_context(
    "notebook", 
    font_scale = 1.5,       
    rc = { 
        "figure.figsize" : (30, 30), 
        "axes.titlesize" : 18 
    }
)

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
from datetime import date
import random 
init_notebook_mode(connected=True)

## Загружаем данные

In [ ]:
data = pd.read_csv('auto_app.csv', delimiter=";",decimal=".", encoding="windows-1251")

In [ ]:
display(data.head(10))

## Проверка пустых значений

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
missing_data(data).head(10)

**Вывол:** Мы видим ТОП-10 полей с наибольшей долей пустых значений. Когда придет время строить  модели машинного обучения, нам придется заполнить эти пропущенные значения. Другим вариантом будет удаление столбцов с большим процентом пропущенных значений, хотя заранее невозможно узнать, будут ли эти столбцы полезны для нашей модели. Поэтому мы пока сохраним все столбцы.

### Проверка типов данных 

Давайте посмотрим на количество столбцов каждого типа данных. int64 и float64 являются числовыми переменными (которые могут быть как дискретными, так и непрерывными). Столбцы объекта содержат строки и являются категориальными признаками.

In [ ]:
data.dtypes.value_counts()

Давайте теперь посмотрим на количество уникальных записей в каждом из (категориальных) столбцов объекта.

In [ ]:
data.select_dtypes(include=['object']).apply(pd.Series.nunique,axis=0)

**Вывод:** Большинство категориальных переменных имеют относительно небольшое количество уникальных записей. Нам нужно будет найти способ справиться с этими категориальными переменными!

## Проверка дисбаланса данных

Давайте посмотрим, как распределяется таргет.

**Таргет** - плохих клиентов определил двумя способами. 
- Клиенты, которые вышли на просрочку 60+
- Клиенты, допустившие просрочку 3 месяца подряд

In [ ]:
data["target"].value_counts()

In [ ]:
temp = data["target"].value_counts()
df = pd.DataFrame({'labels': temp.index,
                   'values': temp.values
                  })
plt.figure(figsize = (6,6))
plt.title('Распределение таргета')
sns.set_color_codes("pastel")
sns.barplot(x = 'labels', y="values", data=df)
locs, labels = plt.xticks()
plt.show()

**Вывод:** Наблюдаем сильный дисбаланс таргета. Это может стать проблемой при построении модели.

## Распределение суммы кредита

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение суммы кредита")
ax = sns.distplot(data["Запрошенная сумма кредита"])

## Распределение суммы ежемесячного платежа

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение суммы ежемесячного платежа")
ax = sns.distplot(data["Сумма ежемесячного платежа"])

## Распределение суммы первоначального взноса

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Распределение суммы первоначального взноса")
ax = sns.distplot(data["Первоначальный взнос"])

**Вывод:** Построение графиков распределений по разным признакам поможет оценить тип распределения, увидеть выбросы.

In [ ]:
def plot_distribution_comp(var,nrow=2):
    
    i = 0
    t1 = data.loc[data['target'] != 0]
    t0 = data.loc[data['target'] == 0]

    sns.set_style('whitegrid')
    plt.figure()
    fig, ax = plt.subplots(nrow,2,figsize=(12,6*nrow))

    for feature in var:
        i += 1
        plt.subplot(nrow,2,i)
        sns.kdeplot(t1[feature], bw=0.5,label="TARGET = 1")
        sns.kdeplot(t0[feature], bw=0.5,label="TARGET = 0")
        plt.ylabel('Density plot', fontsize=12)
        plt.xlabel(feature, fontsize=12)
        locs, labels = plt.xticks()
        plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show();

### Сравнение интервальных значений с TARGET = 1 и TARGET = 0

In [ ]:
var = ['Первоначальный взнос','Сумма ежемесячного платежа','Запрошенная сумма кредита']
plot_distribution_comp(var,nrow=2)

## Обзор данных

In [ ]:
def plot_stats(feature,label_rotation=False,horizontal_layout=True):
    temp = data[feature].value_counts()
    df1 = pd.DataFrame({feature: temp.index,'Количество кредитов': temp.values})

    # Calculate the percentage of target=1 per category value
    cat_perc = data[[feature, 'target']].groupby([feature],as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    
    if(horizontal_layout):
        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))
    else:
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12,14))
    sns.set_color_codes("pastel")
    s = sns.barplot(ax=ax1, x = feature, y="Количество кредитов",data=df1)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    
    s = sns.barplot(ax=ax2, x = feature, y='target', order=cat_perc[feature], data=cat_perc)
    if(label_rotation):
        s.set_xticklabels(s.get_xticklabels(),rotation=90)
    plt.ylabel('Доля таргета = 1 [%]', fontsize=10)
    plt.tick_params(axis='both', which='major', labelsize=10)

    plt.show();

### Пол клиента

Давайте посмотрим на пол клиентов, а также, на отдельном графике, процент кредитов (по полу клиента) со значением TARGET 1 (невозвратный кредит).

In [ ]:
plot_stats('Пол')

**Вывод:** Число клиентов мужского пола почти вдвое превышает число клиентов женского пола. Что касается процента просроченных кредитов, у мужчин больше шансов не вернуть свои кредиты (~ 0,3%), по сравнению с женщинами (~ 0,1%).

### Семейное положение клиента

In [ ]:
plot_stats('Семейное положение',True, True)

**Вывод:** Большинство клиентов состоят в браке, за ними следуют холостые / не состоящие в браке и разведенные.
С точки зрения процента непогашения кредита, вдовцы/вдовы имеет самый высокий процент непогашения (3%), а гражданский брак самый низкий. При этом, вдовы/вдовцы занимают последнее место по количеству выданных кредитов.

### Количество родственников на иждивении, в т.ч. детей до 18 лет

In [ ]:
plot_stats('Количество родственников на иждивении, в т.ч. детей до 18 лет')

**Вывод:**  У большинства клиентов, получающих кредит, нет иждевенцев. Количество кредитов, связанных с клиентами с одним иждевенцем, в 4 раза больше, количество кредитов, связанных с клиентами с тремя иждевенцами; клиенты с 4, 5 и более иждевенцами встречаются гораздо реже.

Что касается погашения, клиенты с 4 иждевенцами имеют наибольший процент не погашения ~0,6%.

### Статус занятости

In [ ]:
plot_stats('Статус занятости',True, False)

**Вывод:** Большая часть кредитов берется клиентами с постоянной занятостью.

Категория с наивысшим процентом невыплаченных кредитов - это работники с постоянной занятостью (более 0,25%).

### Образование клиента

In [ ]:
plot_stats('Образование',True)

**Вывод:** Большинство клиентов имеют высшее образование, затем следуют клиенты с средне спициальным и средним образованием. 

Клиенты с высшим образованием, имеют самую высокую долю невозврата кредита (0,25%).

Таким же образом можно проанизировать и остальные признаки в поиске инсайтов и полезных выводов.

## Аномалии

Одна из проблем, о которой мы всегда хотим знать при EDA, - это аномалии в данных. Это может быть связано с неверно набранными числами, ошибками в измерительном оборудовании, или они могут быть действительными, но крайними измерениями. Один из способов количественного поиска аномалий - это просмотр статистики столбца с использованием метода description. Возьмем для примера признак - **Возраст заявителя**

In [ ]:
(data['Возраст заявителя']).describe()

Максимальное значение составляет 905 лет, а минимальное -5! Здесь явно ошибка в данных.

In [ ]:
data['Возраст заявителя'].plot.hist(title = 'Возраст заявителя');
plt.xlabel('Возраст заявителя');

Просто из любопытства давайте подгруппируем аномальных клиентов и посмотрим, имеют ли они, как правило, более высокие или низкие показатели дефолта, чем остальные клиенты.

In [ ]:
anom = data[data['Возраст заявителя'] == 905]
non_anom = data[data['Возраст заявителя'] != 905]
print('Кредиты с не аномальными данными %0.2f%%' % (100 * non_anom['target'].mean()))
print('Кредиты с аномальными данными %0.2f%% ' % (100 * anom['target'].mean()))

Оказывается, что аномалии имеют более низкий уровень дефолта.

Обработка аномалий зависит от конкретной ситуации, без установленных правил. Один из самых безопасных подходов - просто установить для аномалий недостающее значение, а затем заполнить их (используя импутацию) перед машинным обучением. В этом случае, поскольку все аномалии имеют одинаковое значение, мы хотим заполнить их одинаковыми значениями в случае, если все эти кредиты имеют что-то общее. Кажется, что аномальные значения имеют некоторое значение, поэтому мы хотим сообщить модели машинного обучения, действительно ли мы заполнили эти значения. В качестве решения мы заполним аномальные значения не числом (np.nan), а затем создадим новый логический столбец, указывающий, было ли значение аномальным.

In [ ]:
data['Возраст заявителя (Аномалия)'] = data["Возраст заявителя"] == 905

data['Возраст заявителя'].replace({905: np.nan}, inplace = True)

data['Возраст заявителя'].plot.hist(title = 'Возраст заявителя');
plt.xlabel('Возраст заявителя');

## Корреляция

Один из способов понять данные - найти корреляции между признакми. Мы можем рассчитать коэффициент корреляции Пирсона между каждой переменной и таргетом, используя метод данных .corr.

Коэффициент корреляции - не самый лучший метод для представления «релевантности» объекта, но он дает нам представление о возможных отношениях в данных. Некоторые общие интерпретации абсолютного значения коэффициента корреляции:

1. .00-.19 «очень слабый»
1. .20-.39 «слабый»
1. .40-.59 "умеренный"
1. .60-.79 «сильный»
1. .80-1.0 «очень сильный»

In [ ]:
correlations = data.corr()['target'].sort_values()

print('Самая положительная корреляция:\n', correlations.tail(15))
print('\nСамая отрицательная корреляция:\n', correlations.head(15))

Давайте посмотрим на некоторые из более значимых корреляций: **Процентная ставка** является наиболее позитивной корреляцией. (за исключением TARGET, потому что корреляция переменной с самим собой всегда равна 1!). Корреляция положительная, чем выше процентая ставка, тем с большей вероятностью клиент не выполнит свои обязательства по кредиту.

### Влияние процентой ставки на погашение

In [ ]:
data['Процентная ставка'] = abs(data['Процентная ставка'])
data['Процентная ставка'].corr(data['target'])

In [ ]:
plt.style.use('fivethirtyeight')

plt.hist(data['Процентная ставка'], edgecolor = 'k', bins = 25)
plt.title('Процентная ставка'); plt.xlabel('Процентная ставка'); plt.ylabel('Кол-во');

Само по себе распределение процентной ставки не говорит нам о многом, кроме того, что нет никаких сильных выбросов. Чтобы визуализировать влияние возраста на таргет, мы создадим график оценки плотности ядра (KDE), окрашенный по значению таргета. График оценки плотности ядра показывает распределение одной переменной и может рассматриваться как сглаженная гистограмма (она создается путем вычисления ядра, обычно гауссовского, в каждой точке данных, а затем усреднения всех отдельных ядер для создания единого сглаживания кривая). Для этого графика мы будем использовать Kdeplot.

In [ ]:
plt.figure(figsize = (10, 8))

sns.kdeplot(data.loc[data['target'] == 0, 'Процентная ставка'], label = 'target == 0')

sns.kdeplot(data.loc[data['target'] == 1, 'Процентная ставка'], label = 'target == 1')

plt.xlabel('Процентная ставка'); plt.ylabel('Density'); plt.title('Процентная ставка');

**Вывод:** Кривая target == 1 растет к концу диапазона. Хотя это не является существенной корреляцией (0.035 коэффициент корреляции), эта переменная, вероятно, будет полезна в модели машинного обучения, потому что она действительно влияет на цель

## Парные графики

В качестве заключительного исследовательского графика мы можем создать парный график признаков **Одобренный срок кредита** и **Процентной ставки**. Парный график - отличный инструмент для исследования, потому что он позволяет нам видеть отношения между несколькими парами признаков, а также распределения отдельных признаков. Здесь мы используем библиотеку визуализации seaborn и функцию PairGrid для создания графика пар с диаграммами рассеяния в верхнем треугольнике, гистограммами на диагонали, а также двухмерными графиками плотности ядра и коэффициентами корреляции в нижнем треугольнике.

In [ ]:
ext_data = data[['target', 'Процентная ставка', 'Одобренный срок кредита','Переплата по кредиту','Сумма ежемесячного платежа']]

In [ ]:
age_data = data[['target', 'Процентная ставка']]

In [ ]:
plot_data = ext_data.drop('Процентная ставка', axis=1).copy()

plot_data['Ставка'] = age_data['Процентная ставка']

plot_data = plot_data.dropna().loc[:100000, :]

def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'target', 
                    vars = [x for x in list(plot_data.columns) if x != 'target'])

grid.map_upper(plt.scatter, alpha = 0.2)

grid.map_diag(sns.kdeplot)

grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Парный график', size = 32, y = 1.05);